##  This notebook takes labels from build_training_data/training_images/labels.json and images from build_training_data/training_images/ and uses that to create parking spot car detection models for ParkEz

In [18]:
import os
import json
import time
import numpy as np
import pandas as pd
import cv2
import torch
from torch import nn, optim
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Lambda

In [19]:
with open('build_training_data/training_images/labels.json', 'r') as file:
    human_dict = json.load(file)    
    
spot_keys = list(human_dict[list(human_dict.keys())[0]].keys())
print(spot_keys)

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']


In [5]:
interval = 3
count = 0
total = 0
ordered_keys = list(human_dict.keys())
ordered_keys.sort()
training_keys = []
testing_keys = []

# Model doesn't make a mistake for 
training_keys = ordered_keys[0:170] 
testing_keys = ordered_keys[170:200]
validation_keys = ordered_keys[200:]

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)  
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
        
        # Convolutional layer 3
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU()

        # Convolutional layer 4
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.relu4 = nn.ReLU()

        # Convolutional layer 5
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        # Max pool layer
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 2)

    def forward(self, x):
        # Convolutional layer 1
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.pool(out)

        # Convolutional layer 2
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.pool(out)

        # Convolutional layer 3
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.pool(out)

        # Convolutional layer 4
        out = self.conv4(out)
        out = self.bn4(out)
        out = self.relu4(out)
        out = self.pool(out)

        # Convolutional layer 5
        out = self.conv5(out)
        out = self.bn5(out)
        out = self.relu5(out)
        out = self.pool(out)

        # Flatten for fully connected layer
        out = out.view(out.size(0), -1)

        # Fully connected layer 1
        out = self.fc1(out)
        out = self.dropout(out)

        # Fully connected layer 2
        out = self.fc2(out)
        out = self.dropout(out)

        # Fully connected layer 3
        out = self.fc3(out)

        return out


# Originally in Model_Maker notebook, this preps cropped parking spaces for ML processing
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize((0.5,0.5,0.5,), (0.5,0.5,0.5,))  # Normalize pixel values in the range [-1, 1]
])

In [11]:
def get_models():
    old_models = {}
    for key in spot_keys:
        model_path = os.path.join('Archive', 'old_models', key + '.pth')
        old_models[key] = CNN()
        old_models[key].load_state_dict(torch.load(model_path)) 
        old_models[key].eval() 
    return old_models

def predict_image(to_predict_image, predicting_model_dict):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ret = {}
    for key in spot_keys:
        dir_name = 'input'
        full_path = os.path.join(dir_name, to_predict_image)
        image = Image.open(full_path)
        spots_file_path = os.path.join('spots.json')
        with open(spots_file_path, 'r') as spots_file:
            spots_data = json.load(spots_file)
        x, x_w, y, y_h = spots_data[key]
        cropped_image = image.crop((x, y, x_w, y_h))
        input_tensor = transform(cropped_image)
        input_tensor = input_tensor.unsqueeze(0)
        
        # Move the input tensor to the same device as the model
        input_tensor = input_tensor.to(device)
        
        with torch.no_grad():
            output = predicting_model_dict[key](input_tensor)
            _, predicted = torch.max(output, 1)

        prediction = predicted.item()
        ret[key] = False
        if prediction == 0: ret[key] = True
    return ret

def bulk_predict(keys, predicting_model_dict):
    ret = {}
    for x in keys:
        ret[x] = predict_image(x, predicting_model_dict)
    return ret

In [12]:
class LotImageDataset(Dataset):
    def __init__(self, img_dict, label_dict, transform=None):
        self.img_dict = img_dict
        self.label_dict = label_dict
        self.transform = transform

    def __len__(self):
        return len(self.img_dict)

    def __getitem__(self, idx):
        img_name = list(self.img_dict.keys())[idx]
        image = self.img_dict[img_name]
        label = self.label_dict[img_name]
        if self.transform:
            image = self.transform(image)
        return image, label

In [15]:
def get_dataset_params(my_dict, selected_keys, spot, spot_locs):
    labels_dict = {}
    images_dict = {}
    for to_predict_image in selected_keys:
        dir_name = 'build_training_data/training_images'
        image_path = os.path.join(dir_name, to_predict_image)
        to_crop = Image.open(image_path)
        x, x_w, y, y_h = spot_locs[spot]
        images_dict[to_predict_image] = to_crop.crop((x, y, x_w, y_h))
        pred = 1
        if my_dict[to_predict_image][spot]: pred = 0
        labels_dict[to_predict_image] = pred
    return images_dict, labels_dict

In [14]:
# spots_file_path = os.path.join('Archive', 'old_models', 'spots.json')
spots_file_path = 'build_training_data/spots.json'
with open(spots_file_path, 'r') as spots_file:
    spots_data = json.load(spots_file)



In [16]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


In [17]:
start_time = time.time()  # Save the current time

# Dictionary to store the models
models_es_dict = {}

# Define patience and best loss for early stopping
patience = 5

# Prepare the model for each parking space
for parking_space in spot_keys:
    images, labels = get_dataset_params(human_dict, training_keys, parking_space, spots_data)
    dataset = LotImageDataset(images, labels, transform=transform)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    images, labels = get_dataset_params(human_dict, validation_keys, parking_space, spots_data)
    dataset_validation = LotImageDataset(images, labels, transform=transform)
    dataloader_validation = DataLoader(dataset, batch_size=16, shuffle=True)
    
    # Use the previously defined CNN class
    model = CNN()
    print(f'{parking_space}: Success!!')

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0000025)

    # Define the learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Number of epochs to train for
    num_epochs = 30

    # Move model to GPU if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Initialize best loss for early stopping
    best_loss = np.inf
    no_improve_epochs = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()  # set the model to training mode
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(dataloader):
            # Move data and labels to GPU if available
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Clear the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        train_loss = train_loss / len(dataloader.dataset)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(dataloader_validation):
                # Move data and labels to GPU if available
                if torch.cuda.is_available():
                    inputs = inputs.cuda()
                    labels = labels.cuda()

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

        val_loss = val_loss / len(dataloader_validation.dataset)

        print('Parking Space: {}, Epoch {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(
            parking_space, epoch + 1, train_loss, val_loss))

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            no_improve_epochs = 0
            best_model_state = model.state_dict()  # Save the model state
        else:
            no_improve_epochs += 1

        if no_improve_epochs == patience:
            print('Early stopping due to no improvement after {} epochs.'.format(patience))
            break

        # Step the scheduler
        scheduler.step()

    print('Finished Training for parking space ', parking_space)

    # Save the model after training
    torch.save(best_model_state, f"{parking_space}.pth")

    # Load the best model state and add the model to the dictionary
    model.load_state_dict(best_model_state)
    models_es_dict[parking_space] = model
    
end_time = time.time()  # Save the current time again after the code block has executed
print("Time elapsed: ", ((end_time - start_time)/60), "minutes")

b1: Success!!
Parking Space: b1, Epoch 1, Training Loss: 0.5032, Validation Loss: 0.6773
Parking Space: b1, Epoch 2, Training Loss: 0.2345, Validation Loss: 0.7491
Parking Space: b1, Epoch 3, Training Loss: 0.1567, Validation Loss: 0.4444
Parking Space: b1, Epoch 4, Training Loss: 0.1366, Validation Loss: 0.2150
Parking Space: b1, Epoch 5, Training Loss: 0.1157, Validation Loss: 0.1373
Parking Space: b1, Epoch 6, Training Loss: 0.1131, Validation Loss: 0.0998
Parking Space: b1, Epoch 7, Training Loss: 0.0647, Validation Loss: 0.0664
Parking Space: b1, Epoch 8, Training Loss: 0.0780, Validation Loss: 0.0559
Parking Space: b1, Epoch 9, Training Loss: 0.0643, Validation Loss: 0.0520
Parking Space: b1, Epoch 10, Training Loss: 0.0583, Validation Loss: 0.0451
Parking Space: b1, Epoch 11, Training Loss: 0.0473, Validation Loss: 0.0424
Parking Space: b1, Epoch 12, Training Loss: 0.0680, Validation Loss: 0.0411
Parking Space: b1, Epoch 13, Training Loss: 0.0509, Validation Loss: 0.0420
Parking

KeyboardInterrupt: 